In [1]:
# import packages
!pip install faker
!pip install names
!pip uninstall -y dist/OpenSpecimenAPIconnector-0.9.1-py3-none-any.whl
!pip install dist/OpenSpecimenAPIconnector-0.9.1-py3-none-any.whl

import json
import pandas as pd
import faker
import os
from IPython.display import display
import requests
import random
import ipywidgets as wid
import numpy as np

Found existing installation: OpenSpecimenAPIconnector 0.9.1
Uninstalling OpenSpecimenAPIconnector-0.9.1:
  Successfully uninstalled OpenSpecimenAPIconnector-0.9.1
Processing ./dist/OpenSpecimenAPIconnector-0.9.1-py3-none-any.whl


In [2]:
import OpenSpecimenAPIconnector.os_core as os_core
import OpenSpecimenAPIconnector.os_util as os_util
import OpenSpecimenAPIconnector.mg_core as mg_core
import OpenSpecimenAPIconnector.mg_util as mg_util

In [3]:
def csv_download_link(df, csv_file_name, delete_prompt=False):
    """Display a download link to load a data frame as csv from within a Jupyter notebook"""
    from IPython.core.display import display, HTML
    link = "http://localhost:8888/edit/work/" + csv_file_name
    df.to_csv(csv_file_name, index=False)
    text = "Download/Edit " + csv_file_name
    display(HTML("<a href={} target=_blank>{}</a>".format(link, text)))
   
    if delete_prompt:
        a = input('Press enter to delete the file after you have downloaded it.')
        import os
        os.remove(csv_file_name)

In [4]:
def multi_checkbox_widget(options_dict):
    """ Widget with a search field and lots of checkboxes """
    search_widget = wid.Text()
    output_widget = wid.Output()
    options = [x for x in options_dict.values()]
    options_layout = wid.Layout(
        overflow='auto',
        border='1px solid black',
        width='300px',
        height='300px',
        flex_flow='column',
        display='flex'
    )
    
    #selected_widget = wid.Box(children=[options[0]])
    options_widget = wid.VBox(options, layout=options_layout)
    #left_widget = wid.VBox(search_widget, selected_widget)
    multi_select = wid.VBox([search_widget, options_widget])

    @output_widget.capture()
    def on_checkbox_change(change):
        
        selected_recipe = change["owner"].description
        #print(options_widget.children)
        #selected_item = wid.Button(description = change["new"])
        #selected_widget.children = [] #selected_widget.children + [selected_item]
        options_widget.children = sorted([x for x in options_widget.children], key = lambda x: x.value, reverse = True)
        
    for checkbox in options:
        checkbox.observe(on_checkbox_change, names="value")

    # Wire the search field to the checkboxes
    @output_widget.capture()
    def on_text_change(change):
        search_input = change['new']
        if search_input == '':
            # Reset search field
            new_options = sorted(options, key = lambda x: x.value, reverse = True)
        else:
            # Filter by search field using difflib.
            #close_matches = difflib.get_close_matches(search_input, list(options_dict.keys()), cutoff=0.0)
            close_matches = [x for x in list(options_dict.keys()) if str.lower(search_input.strip('')) in str.lower(x)]
            new_options = sorted(
                [x for x in options if x.description in close_matches], 
                key = lambda x: x.value, reverse = True
            ) #[options_dict[x] for x in close_matches]
        options_widget.children = new_options

    search_widget.observe(on_text_change, names='value')
    display(output_widget)
    return multi_select

In [5]:
def display_search_select_box(cps_dict):
    descriptions = [key for key in  cps_dict.keys()]

    options_dict = {
        x: wid.Checkbox(
            description=x, 
            value=False,
            style={"description_width":"0px"}
        ) for x in descriptions
    }

    def f(**args):

        results = [key for key, value in args.items() if value]
        display(results)

    ui = multi_checkbox_widget(options_dict)
    out = wid.interactive_output(f, options_dict)
    display(wid.HBox([ui, out]))
    return ui

In [6]:
# The URL of the Openspecimen
print('Fill in Openspecimen url (http(s)://<IP-Adress>:<port>) and press enter')
base_url = 'http://biobank-7-2.silicolab.bibbox.org' 
base_url +='/openspecimen/rest/ng'
print('The whole URL for API Calls is: ' + base_url)

Fill in Openspecimen url (http(s)://<IP-Adress>:<port>) and press enter
The whole URL for API Calls is: http://biobank-7-2.silicolab.bibbox.org/openspecimen/rest/ng


In [7]:
# Loginname and Pasword of the API-User, recommended is an extra API User
print('Loginname of the user:')
loginname='admin'#input() 
print('Password of the user:')
password='Login@123'#input()
auth = (loginname, password)

Loginname of the user:
Password of the user:


In [8]:
# get necessary cp information
cp_tools = os_core.collection_protocol(base_url, (loginname, password))

# get cp info and wirte a dict in order to simplify the api output
cps = cp_tools.get_all_collection_protocols()
cps_dict = {}
for i, item in enumerate(cps):
    # print("Cp Title:")
    # print(item["title"])
    # print("Cp ID: ",item["id"], "\n")
    cps_dict[item["title"]] = item["id"]

In [9]:
# select collection protocolls
ui = display_search_select_box(cps_dict)

Output()

In [10]:
# get selected cp_ids
cp_ids = []
selected_options = [widget.description for widget in ui.children[1].children if widget.value]
for item in selected_options:
    cp_ids.append(cps_dict[item])
print(cp_ids)

[17, 18]


In [11]:
exporter = mg_util.bbmri_connector()
bbmri_file = exporter.execute(cp_ids)

URLError: <urlopen error [Errno 101] Network is unreachable>

In [ ]:
# sheetnames:
bb_sheet = "eu_bbmri_eric_biobanks"
per_sheet = "eu_bbmri_eric_persons"
cp_sheet = "eu_bbmri_eric_collections"

output_bb = wid.Output()
output_per = wid.Output()
output_cp = wid.Output()

tab = wid.Tab([output_bb, output_per, output_cp])
tab.set_title(0, bb_sheet)
tab.set_title(1, per_sheet)
tab.set_title(2, cp_sheet)
display(tab)
with output_bb:
    display(bbmri_file[bb_sheet])
with output_per:
    display(bbmri_file[per_sheet])
with output_cp:
    display(bbmri_file[cp_sheet])


